# Homework 4 - Group 10

## Our team:

- Giulia Scikibu Maravalli
- Katsiaryna Zavadskaya
- Sara Cordaro 

*Libraries*:

In [15]:
# libraries
from requests import get
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
import nltk # to clean the file
from nltk.stem import PorterStemmer # for stemming
import math
import numpy as np
import string
from collections import defaultdict
import json
import numpy as np
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import pickle

## 1) Does basic house information reflect house's description?  
In this assignment we will perform a clustering analysis of house announcements in Rome from [Immobiliare.it](https://www.immobiliare.it/).  

### Scraping  & Datasets

Retrieving at least 10k announcements from *Immobiliare.it*, we created our dataset. Let's see how!  

In our **main function**, we connect to the website (with our function **html_soup**, where we used *Beautiful Soup* library) and we iterate through its pages (every page contain 25 announcements) until we have processed 10 thousand announcements. Every time we pick an announcement we perform the following operations:

- check if all the information we need are present (i.e. price, local, bathroom, floor, surface), otherwise we skip this announcement, and go to the next. To take these information we used **take_info** function.  


- if we find the information above, we take the link of this announcement, and verify if there is the description (if not the house has been removed and we can go over it as well). To take description we used **take_description** function.


- after we took the description, we processed it as a string: remove escape, punctuation and italian stopwords(this operation is executed by **clean_text** function). At the end of this cleaning, we save the modified description inside a new file, called *desc_n.txt* (where *n* is the number of the announcement). (**desc_file** function will do this).


- At this point, if the announcement has both the required fields and description, we save every field in a dedicated list (look at the chunck of global variables, we create a list for each field).  

And finally, we have our dataframes.

In [2]:
# global variables

# seconds to wait, to prevent the website block
t = .400

# url main
immobiliare_link = 'https://www.immobiliare.it'

# total number of announcements
N = 10000

# list of announcement's links
link = []
# list of announcement's prices
price = []
# list of announcement's locals
local = []
# list of announcement's surfaces
surface = []
# list of announcement's bathrooms
bathroom = []
# list of announcement's floors
floor = []
# list of announcement's numbers
number = []
# list of number of words
doc_num_words = []

# number of announcements
num_announcement = 1
# page of announcements
page = 1


# dictionary (key: word, value: list of idx repeat for the number of time that the word appears)
dic_words = defaultdict(list)

# dictionary (key: word, value: tfidf for each announcement)
tfidf_dic = defaultdict(list)

# escape
escape = r'\n'
# set punctuation
set_punt = list(string.punctuation)
# set stopwords
stop_words = set(nltk.corpus.stopwords.words('italian'))
# stemming
ps = PorterStemmer()
# main path
main_path = 'C:/Users/giuli/Desktop/AMD_HW4'

In [3]:
# function to clean the description (to remove punctuation and /n)
def clean_text(text):
    text = text.replace(escape, ' ')
    # split description in a list
    list_txt = text.split(' ')
    # list of clean words
    new_words = []
    # iterate each word of description
    for word in list_txt:
        if word.isalpha():
            if word not in set_punt:
                # take each char in the string
                for c in word:
                    # delete char if it is a punctuation
                    if c in set_punt:
                        word = word.replace(c,"")
                # save the string only if it isn't a stop words
                if word not in stop_words: 
                    # we save the word in lowercase - so it is easier work in this way
                    word = word.lower()
                    new_words.append(word)
    # convert description in strings
    new_desc = ' '.join(new_words)
    return new_desc
        

In [4]:
def desc_file(idx,text):
    text = clean_text(text)
    # create a new folder where we save cleaned file
    if not os.path.exists(main_path):
        os.mkdir(main_path)
    # create a file
    with open(main_path + "/desc_" +str(idx) + ".txt", "w") as f:
        # write description
        f.write(text)
    return

In [5]:
# html document
def html_soup(url):
    # request the server the content of the web page and store the server’s response - html document
    while True:
        try:
            response = get(url)
            html_soup = BeautifulSoup(response.text, 'html.parser')
            return html_soup
        except:
            time.sleep(t)
            continue

In [6]:
# take description of each valid announcement
def take_description(url_home,num):  
    try:
        html_home = html_soup(url_home)
        desc_home = html_home.find('div', class_ ="col-xs-12 description-text text-compressed").text
        desc_file(num,desc_home)
        return True
    except:
        return False

In [7]:
def take_info(announcement_info):
    # try if the informations are not NONE
    try:
        infos = []
        # take price of the home
        price_info = announcement_info.find('li', class_="lif__item").text
        # clean the price
        price_info = price_info.strip()        
        if price_info.count("€") >= 2:
            price_info = "€ " + ''.join(price_info.split("€ ", 2)[:2])
        # verify if it is only one home or a group of home
        if 'da' not in price_info:
            infos.append(price_info)
            # take the other informations
            all_info = announcement_info.find_all('div', class_ = "lif__data")
            # save the other informations
            for info in all_info[:-1]:
                infos.append(info.span.text)
            # take the home's floor    
            info_floor = all_info[-1].find('abbr')['title']
            infos.append(info_floor)
            return infos
    # if there are some informations NONE    
    except:
        infos = []
        return infos



In [8]:
#MAIN FUNCTION

start_time = time.time()


# we want to analyze the info's on https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1
# to retrieve at least 10k announcements
while num_announcement <= N:
    # assign the address of the web page to a variable named url
    url = immobiliare_link + '/vendita-case/roma/?criterio=rilevanza&pag=' + str(page)
    html_announcements = html_soup(url)    
    # exctract all informations for each announcement
    announcements_info = html_announcements.find_all('ul', class_ ="listing-features list-piped")
    # extract all the div containers that have a class attribute of listing-item_body--content - result set
    announcements_link = html_announcements.find_all('div', attrs={'class':'listing-item_body--content'})
    # iterate each list of home's informations
    for index in range(len(announcements_info)):
        # take information of announcement
        infos = take_info(announcements_info[index])
        # check if there are all informations
        if infos != None and (len(infos) == 5):
            # take the home's link
            home_link = announcements_link[index].a['href']
            if immobiliare_link not in home_link:
                home_link = immobiliare_link + home_link
            # take description link of a home and control if the home is valid
            control = take_description(home_link, num_announcement)
            # if the home has the description is valid
            if control == True:
                # save all informations of a valid home in the global variables
                link.append(home_link)
                price.append(infos[0])
                local.append(infos[1])
                surface.append(infos[2])
                bathroom.append(infos[3])
                floor.append(infos[4])
                number.append(int(num_announcement))
        num_announcement += 1
    page += 1
    
end_time = time.time()
print(end_time - start_time)    

6348.990574359894


In [9]:
# create dataframe 1
df_1 = pd.DataFrame({'Price': price,
                       'Local': local,
                       'Surface': surface,
                       'Bathroom': bathroom,
                       'Floor': floor}, index = number)

In [10]:
df_1.head()

,Price,Local,Surface,Bathroom,Floor
2,€ 225.000,2,50,1,1
3,€ 400.000,3,60,1,3
4,€ 500.000,3,89,2,3
5,€ 574.000,4,89,2,5
6,€ 300.000,2,46,1,4


We saved the dataframe with all required fields to a .csv file:

In [11]:
# save the dataframe 1 in a file csv
df_1.to_csv(main_path+'/matrix_1.csv')

Since we want to build from this dataframe a real matrix, we need to filter it:

In [18]:
# filter dataframe to create matrix_1

# prices clean
price_clean = []
for p in price:
    p = p.replace('€ ','')
    p = p.replace(escape,'')
    p = p.replace('.','')
    price_clean.append(p)
    
# locals clean 
local_clean = []
for l in local:
    l = l.replace('+','')
    local_clean.append(int(l))

# surfaces clean
surface_clean = []
for s in surface:
    s = s.replace('.','')
    surface_clean.append(int(s))

# bathrooms clean
bath_clean = []
for b in bathroom:
    b = b.replace('+','')
    bath_clean.append(int(b))

data_1 = pd.DataFrame({'Price': price_clean,
                       'Local': local_clean,
                       'Surface': surface_clean,
                       'Bathroom': bath_clean,
                       'Floor': floor})

data_1 = data_1[data_1.Price.apply(lambda x: x.isdigit())]
data_1 = data_1[data_1['Local'].apply(lambda x: (str(x)).isdigit())]
data_1 = data_1[data_1['Surface'].apply(lambda x: str(x).isdigit())]
data_1 = data_1[data_1['Bathroom'].apply(lambda x: (str(x)).isdigit())]
data_1 = data_1[data_1['Floor'].apply(lambda x: (str(x)).isdigit())]

In [19]:
data_1.head()

,Price,Local,Surface,Bathroom,Floor
0,225000,2,50,1,1
1,400000,3,60,1,3
2,500000,3,89,2,3
3,574000,4,89,2,5
4,300000,2,46,1,4


In [20]:
# create matrix 1
matrix_1 = data_1.values
matrix_1 = np.matrix(matrix_1).astype(float)

We save the obtained matrix into a .csv file:

In [22]:
# save the matrix 1 in a file txt
np.savetxt('last_matrix_1.txt', matrix_1, delimiter=",")

Now after we have built the first dataframe, we need to create the second one. In order to do so, we first iterate through all the *desc_n.txt* created before and for each of them, we stem all the words in the file and store the n. of words in *doc_num_words* list. At the end of iteration we create a new file, called *vocabulary.txt* where we save a dictionary containing as keys the word and as value a list that have the index of the announcement in which that word appeared (if the word appears muliple times in an announcement, that index will be repeated). 

In [12]:
# create vocabularies
# iterate the files
for idx in number:
    with open(main_path + "/desc_" +str(idx)+ ".txt") as f:
        # split the line according to '\t'
        line = f.read()
        list_file = line.split(' ')
        # save the word as key and the index of the file in list of values
        for word in list_file:
            # stemming
            word = ps.stem(word)
            dic_words[word].append(idx)
        num_words = len(list_file)
        doc_num_words.append(num_words)
# save dictionary in a file
with open(main_path +'/vocabulary.txt', 'w') as f:
        f.write(json.dumps(dic_words))

Now we compute *tf-Idf* and built the second matrix in the format requested:

In [13]:
# function that returns the tfIdf (announcement, word)
def compute_tfIdf(word, idx , voc):
    # tf is term frequences on a determinate description
    tf =(voc[word].count(number[idx]))/(doc_num_words[idx])
    # df document frequences
    df = len(set(voc[word]))
    # idf inverse document frequency
    idf = math.log10(len(number)/df)
    return(tf*idf)

In [14]:
# open and save vocabulary
vocabulary = json.load(open(main_path +'/vocabulary.txt', 'rb'))
list_words = sorted(list(vocabulary.keys()))
for word in list_words:
    for idx in range(len(number)):
        tfidf = compute_tfIdf(word,idx,vocabulary)
        tfidf_dic[word].append(tfidf)

We store the result in a dataframe, and then save it as .csv and .txt (as a matrix) files.

In [15]:
# create dataframe 2
df_2 = pd.DataFrame(data = tfidf_dic, index = number)
    

In [16]:
df_2.head()

,a,aabbiamo,ab,abacu,abamelek,abano,abb,abba,abbaini,abbaino,...,élégant,énergétiqu,équipé,étage,état,étude,été,ìntegrata,último,über
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# save the dataframe 2 in a file csv
df_2.to_csv(main_path+'/matrix_2.csv')

In [21]:
# create matrix 2
matrix_2 = df_2.values
matrix_2 = np.matrix(matrix_2).astype(float)

In [23]:
# save the matrix 2 in a file txt
np.savetxt('last_matrix_2.txt', matrix_2, delimiter=",")

We also save the list of announcement's number beacuse it will be useful in next steps.

In [17]:
#save list of announcement's number
with open('list', 'wb') as fp:
    pickle.dump(number, fp)

## 2) Find the duplicates!
Given [passwords2.txt](https://drive.google.com/file/d/1wTmOU-yqk4qdQYg42AquhzgpNGrRA96d/view) file as input, our goal is to find how many duplicates are present inside it. At first, we consider duplicates not only equal strings, but also strings that contain the same characters (i.e. 'AABA' = 'AAAB'), further our check is case sensitive, thus 'AaBa' $\neq$ 'AAba', if you want to make our code unsensitive about lower/uppercase you just need to uncomment in the definition of our *hashCode()* function this line: *sorted_chars[i] = sorted_chars[i].lower()* and run it.

We proceeded in this way:

- iterate through every line in the file (each line is a string of 20 characters).

- trasform each string in a list of strings (e.g. 'AABA' -> \['A','A','B','A'\]).

- sort the list, in this way we made equal  words with same charcter in different order (now characters are in the same order).

- then we iterate through every character, and with the function **convertToNumber** we convert the character to a number and then to have a very large numeber, we multiplied that integer to a high number: 17^esp. Then we sum the value of each character of that string. In this way we transform the string into a large number (take a look to **hashCode** function defined below).  
Note: *esp* is the index of the character, right now this not particularly important since we sort the order of the characters (but in the second part of this exercize, when we do not sort them, it will allow us to differenciate bewtween word with same character in different order).

- with **HashMap** function we did the modulus between our large number (converted string) and 100011723599, a large prime number. We decided to use this value because as large number it increases the number of possible results of the modulus and as prime number it allows us to diversify as much as possible the outputs.

- we created a bitarray of length 100011723599, composed by zero (i.e. an array of 100011723599 zeros).

- agan with *HashMap* we took the result of the modulus, go to that position in the bitarray, and change the zero to one, if in that position there is already a one, it means that another string had (after the process above) the same value (it is a duplicate!!!). If that is the case we add 1 to a counter, the variable *duplicates*. 

- at the end of iteration we check the value of *duplicates*, intuitively it will tell us the number of duplicates inside the file. 

In [7]:
def convertToNumber(s):
    return int.from_bytes(s.encode(), 'little')

def hashCode(pw):
    # list of chars in the password
    chars = list(pw)
    # doesn't matter the order of the password
    sorted_chars= sorted(chars)
    # new num associated to the pw
    pw_num = 0
    for i in range(20):
        #uncomment line below if you do not want a case sensitive approach
        #sorted_chars[i] = sorted_chars[i].lower()
        w_encode = convertToNumber(sorted_chars[i])
        exp = 19 - i
        pw_num += w_encode * (17**exp)
    return pw_num

prime_n = 100011723599
# initialize bitarry length prime_n 
bit_array = bitarray(prime_n)
bit_array.setall(0)

def hashMap(pw_num):
    count = 0
    index = abs(pw_num)%prime_n
    if (bit_array[index] == 0):
        bit_array[index] = 1
    else:
        count = 1
    return count

# iterate passwords in the file.txt
duplicates = 0

with open('passwords2.txt') as pw_file:
    for pw in pw_file:
        pw_num = hashCode(pw)
        duplicates += hashMap(pw_num)

duplicates

10049831


We get a result very close to 10 milion (the real number of duplicate in the file). However we have some *false positives*, 49831. 

#### Now we do the same thing as before, but taking into account the order of the characters.  
We use the same code of the previuos part, but this time we do not sort the list of character in the string, then word with same character in different position will remain different. As said in the note above, after tranforming the character to an integer we multiply it to 17^*esp*, *esp* is the position of the character inside the string, thanks to this different word with same character will have different numeric values. For the rest, the code remains unchanged.

In [9]:
prime_n = 100011723599
# initialize bitarry length prime_n 
bit_array = bitarray(prime_n)
bit_array.setall(0)

def hashCode(pw):
    # list of chars in the password
    chars = list(pw)
    # new num associated to the pw
    pw_num = 0
    for i in range(20):
        #uncomment line below if you do not want a case sensitive approach
        #chars[i] = chars[i].lower()
        w_encode = convertToNumber(chars[i])
        exp = 19 - i
        pw_num += w_encode * (17**exp)
    return pw_num

# iterate passwords in the file.txt
duplicates1 = 0

with open('passwords2.txt') as pw_file:
    for pw in pw_file:
        pw_num = hashCode(pw)
        duplicates1 += hashMap(pw_num)

duplicates1 

5055169

There should be around 50 thousand duplicates, taking into account the order of characters.